# Modeling New Waves

Unfortunately, viruses won't adhere to the exponential decline portion of our model and cease to spread. Viruses typically have multiple periods of exponential growth and decline. Our current generalized logistic function can only capture a single "wave" of growth and decline.

## Limit the data

One solution to this problem is to limit the number of previous days that our model trains on. For instance, we could build a model based on just the last 50 days worth of data. The model simply "forgets" about all data before this period, making it able to continually capture new waves in the future regardless of how many occur.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prepare import PrepareData
from solutions import predict_all
plt.style.use('dashboard.mplstyle')

def general_logistic(x, L, x0, k, v):
    return L / ((1 + np.exp(-k * (x - x0))) ** (1 / v))

### Concrete example

Let's walk through a concrete example to see how this works. Take a look at the cases from Australia through September 25, 2020. You'll notice two distinct waves, one beginning in the second half of March and another beginning in July.

In [ ]:
data = PrepareData(download_new=False).run()
area = "Australia"
cases = data['world_cases'][area][:'2020-09-25']
cases.plot(title=area);

Attempting to model all of this data using the generalized logistic model yields poor results. Here, we use August 1st as the last day of training data.

In [ ]:
last_date = "2020-08-01"
L_min, L_max = cases[last_date], 100_000
x0_min, x0_max = -50, 50
k_min, k_max = 0.01, 0.5
v_min, v_max = 0.01, 2
lower = L_min, x0_min, k_min, v_min
upper = L_max, x0_max, k_max, v_max
bounds = lower, upper
p0 = L_min * 2, 0, 0.1, 0.1
predict_all(cases, start_date=None, last_date=last_date, n_smooth=15, n_pred=50, 
            model=general_logistic, bounds=bounds, p0=p0, title="Australia");

If we select data beginning after the first wave, then the generalized logistic model should work.

In [ ]:
start_date = '2020-06-01'
cases[start_date:].plot(title='Australia from Jun 1 - Sep 25');

Note, that this is the first time we've used something other than `None` for the `start_date`.

In [ ]:
predict_all(cases, start_date=start_date, last_date=last_date, n_smooth=15, n_pred=50, 
            model=general_logistic, p0=p0, bounds=bounds, title="Australia");

### What went wrong?

Unfortunately, our model produced an obviously wrong curve. The issue here is that the left tail of the curve is very flat, but does not begin at 0 like the initial data. Our model has the ability to shift the curve horizontally using `v`, but not vertically. Let's add a new parameter, `s`, to the model that shifts the data vertically. We'll allow it to range from 0 up to the value of the last day.

$$f(x) = \frac{L - s}{(1 + e^{-k(x - x_0)})^{\frac{1}{v}}} + s$$


In [ ]:
def general_logistic_shift(x, L, x0, k, v, s):
    return (L - s) / ((1 + np.exp(-k * (x - x0))) ** (1 / v)) + s

start_date = "2020-6-1"
last_date = "2020-08-01"
k_min, k_max = 0.01, 0.5
v_min, v_max = 0.01, 2
L_min, L_max = cases[last_date], 100_000

s_min, s_max = 0, cases[-1]
x0_min, x0_max = -50, 50
lower = L_min, x0_min, k_min, v_min, s_min
upper = L_max, x0_max, k_max, v_max, s_max
bounds = lower, upper
p0 = L_min * 5, 0, 0.1, 0.1, s_min
params, y_pred = predict_all(cases, start_date=start_date, last_date=last_date, n_smooth=15,
            n_pred=50, model=general_logistic_shift, p0=p0, bounds=bounds, 
            title="Australia - Shifted Generalized Logistic");

In [ ]:
params

While the model did not make an accurate prediction, it was able to capture the shape of the curve.

## Defining functions to create the limits and bounds

The upper limit of the curve (`L_max`) and its initial guess can have great influence on the trajectory of the fitted curve. For instance, if we change the maximum value of L from 100,000 to 30,000, we get a drastically different curve.

In [ ]:
L_max = 30_000
lower = L_min, x0_min, k_min, v_min, s_min
upper = L_max, x0_max, k_max, v_max, s_max
bounds = lower, upper
p0 = L_max, 0, 0.1, 0.1, 0
predict_all(cases, start_date=start_date, last_date=last_date, n_smooth=15, 
            n_pred=50, model=general_logistic_shift, p0=p0, bounds=bounds, 
            title="Australia - L_max of 30,000");

Of course, there is no way to know ahead of time which value for `L_max` will show superior results, but we still need to have a systematic way of choosing a value for it and its initial guess for every area. A simple idea involves finding the approximate daily percentage increase of the cumulative total on the last known date. We can use it as a rate of growth for the next 30 to 100 days. Let's see this on the smoothed data by calculating the daily percent change on the cumulative total.

In [ ]:
from solutions import smooth
s_smooth = smooth(cases[:last_date], n=15)
s_smooth_pct = s_smooth.pct_change()
s_smooth_pct.tail()

Plotting this data shows a better picture of the percentage growth.

In [ ]:
s_smooth_pct[start_date:].plot();

Australia shows about 3.5% growth in the last day. Using this value of growth for the next 50 days would yield about 102,000 cases as the upper bound. Making an estimation like this is just an alternative and simpler way of using exponential growth.

In [ ]:
s_smooth[-1] * (1 + s_smooth_pct[-1]) ** 50

The function below accepts the smoothed series of data, finds the last percentage change and uses it to estimate the L bounds the initial guess, and the midpoint between the two. In the event that an area has 0 cases, the percent change will return a missing value, so we set it 

In [ ]:
def get_L_limits(s, n1, n2):
    """
    Finds the min and max bounds for L and its initial guess

    Parameters
    ----------
    s : smoothed Series

    n1, n2 : min and max days of exponential growth

    Returns
    -------
    three-item tuple - min/max L bounds and initial
    """
    last_val = s[-1]
    last_pct = s.pct_change()[-1] + 1
    L_min = last_val * last_pct ** n1
    L_max = last_val * last_pct ** n2 + 1
    L0 = (L_max - L_min) / 2 + L_min
    if np.isnan(L_min):
        L_min, L_max, L0 = 0, 1, 0
    return L_min, L_max, L0

Let's use it to get the bounds and initial guess.

In [ ]:
get_L_limits(s_smooth, 5, 50)

We can write one more function to return both the bounds and initial point 

In [ ]:
def get_bounds_p0(s, n1=5, n2=50):
    """
    Finds the bounds and initial guesses for each
    parameter of the shifted logistic function

    Parameters
    ----------
    s : smoothed Series

    n1, n2 : min and max days of exponential growth

    Returns
    -------
    two-item tuple - bounds and p0
    """
    L_min, L_max, L0 = get_L_limits(s, n1, n2)
    x0_min, x0_max = -50, 50
    k_min, k_max = 0.01, 0.1
    v_min, v_max = 0.01, 2
    s_min, s_max = 0, s.iloc[-1] + 0.01
    s0 = s_max / 2
    lower = L_min, x0_min, k_min, v_min, s_min
    upper = L_max, x0_max, k_max, v_max, s_max
    bounds = lower, upper
    p0 = L0, 0, 0.1, 0.1, s0
    return bounds, p0

In [ ]:
bounds, p0 = get_bounds_p0(s_smooth)
bounds

In [ ]:
p0

## Finding new waves

Not every country will have perfectly separated waves like Australia. Fortunately, it's not necessary to pick the exact start date of a new wave. Our model is now capable of shifting vertically and horizontally, so will be able to capture the beginning, middle, and tail of the curves regardless of where the starting point is. 

Here, we use a start date of July 20, well past the beginning of the initial exponential increase in Australia. We allow the model to see data up to August 20, when it has entered into exponential decline. This time the model makes an accurate prediction. Modeling typically becomes easier as the curve approaches the tail.

In [ ]:
start_date = "2020-07-20"
last_date = "2020-08-20"
bounds, p0 = get_bounds_p0(cases[:last_date])
params = predict_all(cases, start_date=start_date, last_date=last_date, n_smooth=15, 
                     n_pred=30, model=general_logistic_shift, bounds=bounds, p0=p0,
                     title="Australia - modeled from the middle of a wave")

Here, we select cases in Italy from March 20th to April 10th as our training data and build a model to make predictions over the next 30 days. Again, our model is capable of fitting data beginning at any point during its wave.

In [ ]:
area = 'Italy'
start_date = '2020-03-20'
last_date='2020-04-10'
cases = data['world_cases'][area]
bounds, p0 = get_bounds_p0(cases[:last_date])
params = predict_all(cases, start_date=start_date, last_date=last_date, n_smooth=15,
                     n_pred=30, model=general_logistic_shift, p0=p0, bounds=bounds, 
                     title=area)

## Summarizing the final model

The final model that we'll use for predicting cases is a generalized logistic function with upper asymptote `L`, growth rate `k`, horizontal shift `x0`, asymmetric control `v`, and vertical shift `s`. The data is smoothed beforehand using LOWESS. Only recent data is used during training in order to capture new waves.

$$f(x) = \frac{L - s}{(1 + e^{-k(x - x_0)})^{\frac{1}{v}}} + s$$